In [1]:
import json
import pandas as pd
from utils import preprocessing, set_index
from matplotlib import pyplot as plt
import numpy as np
from tensorflow.keras import models
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
from dc_vae_continual import DCVAE, Sampling
from utils import samples_conditions_embedd
import pickle
from statsmodels.graphics.tsaplots import plot_acf

2023-06-18 22:47:14.854995: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


### Settings

In [ ]:
settings_path = 'settings/model_settings_continual_J4.txt'
settings = json.load(open(settings_path, 'r'))
print(settings)

### Train data

In [ ]:

#path = '../../Datasets/TELCO/'
path = "/home/gastong/Documentos/TELCO/v1/" #Rosaluna
filenames_train = ["TELCO_data_2021_01.zip", 
                #"TELCO_data_2021_02.zip", "TELCO_data_2021_03.zip",
                #"TELCO_data_2021_04.zip", "TELCO_data_2021_05.zip", "TELCO_data_2021_06.zip",
                #"TELCO_data_2021_07.zip", "TELCO_data_2021_08.zip", "TELCO_data_2021_09.zip",
                #"TELCO_data_2021_10.zip", "TELCO_data_2021_11.zip", "TELCO_data_2021_12.zip"
                ]
files_train = [path+ i for i in filenames_train]

# Load data and predictions
# Read CSV files from List
data_train = pd.concat(map(pd.read_csv, files_train))
data_train = set_index(data_train)
data_train = preprocessing(data_train, flag_scaler=False, outliers=True)
data_train.head()

In [ ]:
#define number of rows and columns for subplots
nrow=12
ncol=1

columns_list = data_train.columns

time_start = '2021-01-01'
time_end = '2021-12-31'
fig, axes = plt.subplots(nrow, ncol, figsize=(20,30), constrained_layout=True, sharex=True)

# plot counter
count=0

for r in range(nrow):
    for c in range(ncol):
        data_train[columns_list[count]][time_start:time_end].plot(ax=axes[r])
        axes[r].set_title(columns_list[count])
        count+=1

### Test data

In [ ]:
#path = '../../Datasets/TELCO/'
path = "/home/gastong/Documentos/TELCO/v1/" #Rosaluna
filenames_test = ["TELCO_data_2022_01.zip", "TELCO_data_2022_02.zip", "TELCO_data_2022_03.zip", "TELCO_data_2022_04.zip"]
files_test = [path+ i for i in filenames_test]

In [ ]:
# Load data and predictions
# Read CSV files from List
data_test = pd.concat(map(pd.read_csv, files_test))
data_test = set_index(data_test)
data_test = preprocessing(data_test, flag_scaler=False, outliers=True)
data_test.head()

In [ ]:
#define number of rows and columns for subplots
nrow=12
ncol=1

columns_list = data_test.columns

time_start = '2022-01-01'
time_end = '2022-04-30'
fig, axes = plt.subplots(nrow, ncol, figsize=(20,30), constrained_layout=True, sharex=True)

# plot counter
count=0

for r in range(nrow):
    for c in range(ncol):
        data_test[columns_list[count]][time_start:time_end].plot(ax=axes[r])
        axes[r].set_title(columns_list[count])
        count+=1

## Predicciones

In [ ]:
# Model initialization
model = DCVAE(
    settings['T'],
    settings['M'],
    settings['cnn_units'],
    settings['dil_rate'],
    settings['kernel'],
    settings['strs'],
    settings['batch_size'],
    settings['J'],
    settings['lr'],
    settings['lr_decay'],
    settings['decay_rate'],
    settings['decay_step'],
    settings['time_embedding'],
    settings['summary'],
    ) 

In [ ]:
anom_train, error_train, reconst_train, sig_train, latent_space = model.predict(data_train,
                                        True,
                                        large_result=True,
                                        load_alpha=False,
                                        alpha_set_up = list(np.ones(12)),
                                        alpha_set_down = list(np.ones(12)),
                                        name=settings["model_name"]
                                                    )
th_up_train = reconst_train + sig_train
th_down_train = reconst_train - sig_train

**Series reales con las predicciones de media y sigma**

In [ ]:
#define number of rows and columns for subplots
nrow=12
ncol=1
data_aux = data_train[reconst_train.index[0]:].copy()

columns_list = data_aux.columns

time_start = '2021-01-01'
time_end = '2021-01-30'
fig, axes = plt.subplots(nrow, ncol, figsize=(20,30), constrained_layout=True, sharex=True)

# plot counter
count=0

for r in range(nrow):
    for c in range(ncol):
        data_aux[columns_list[count]][time_start:time_end].plot(ax=axes[r])
        reconst_train[columns_list[count]][time_start:time_end].plot(ax=axes[r])
        axes[r].fill_between(data_aux[columns_list[count]][time_start:time_end].index,
                         th_up_train[columns_list[count]][time_start:time_end],
                         th_down_train[columns_list[count]][time_start:time_end],
                         facecolor='orange', alpha=0.6, interpolate=True)
        axes[r].set_title(columns_list[count])
        count+=1

In [ ]:
anom_test, error_test, reconst_test, sig_test, latent_space = model.predict(data_test,
                                        True,
                                        large_result=True,
                                        load_alpha=False,
                                        alpha_set_up = list(np.ones(12)),
                                        alpha_set_down = list(np.ones(12)),
                                        name=settings["model_name"]
                                                    )
th_up_test = reconst_test + sig_test
th_down_test = reconst_test - sig_test

In [ ]:
#define number of rows and columns for subplots
nrow=12
ncol=1
data_aux = data_test[reconst_test.index[0]:].copy()

columns_list = data_aux.columns

time_start = '2022-01-01'
time_end = '2022-04-30'
fig, axes = plt.subplots(nrow, ncol, figsize=(20,30), constrained_layout=True, sharex=True)

# plot counter
count=0

for r in range(nrow):
    for c in range(ncol):
        data_aux[columns_list[count]][time_start:time_end].plot(ax=axes[r])
        reconst_test[columns_list[count]][time_start:time_end].plot(ax=axes[r])
        axes[r].fill_between(data_aux[columns_list[count]][time_start:time_end].index,
                         th_up_test[columns_list[count]][time_start:time_end],
                         th_down_test[columns_list[count]][time_start:time_end],
                         facecolor='orange', alpha=0.6, interpolate=True)
        axes[r].set_title(columns_list[count])
        count+=1

## Espacio de latencia

In [ ]:
if settings["time_embedding"]:
    X, time_info = samples_conditions_embedd(data_train, settings["T"])  
    input_samples_train = (X, time_info)
else:
    X = data_train.values
    N = data_train.shape[0]
    X = np.array([X[i: i + settings["T"]] for i in range(0, N - settings["T"]+1)])
    input_samples_train = X

z_mu_train, z_log_var_train, z_train = model.encoder.predict(input_samples_train)
z_sig_train = np.exp(0.5*z_log_var_train)

### Ejemplo de como se agrupan los puntos por hora

### $\mu_z$

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(15,15), constrained_layout=True)

for i in range(4):
    for j in range(4):
        axes[i,j].scatter(z_mu_train[:, -1, i], 
                          z_mu_train[:, -1, j], 
                          c=data_train.index.hour[(settings['T']-1):])

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(15,15), constrained_layout=True)

day_number = 288
count = 4
for i in range(4):
    for j in range(4):
        axes[i,j].scatter(z_mu_train[count*day_number:(count+1)*day_number, -1, i], 
                          z_mu_train[count*day_number:(count+1)*day_number, -1, j], 
                          c=data_train.index.hour[(settings['T']-1)+count*day_number:(settings['T']-1)+(count+1)*day_number])

### $\sigma_z$

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(15,15), constrained_layout=True)

day_number = 288
count = 4
for i in range(4):
    for j in range(4):
        axes[i,j].scatter(z_sig_train[:, -1, i], 
                          z_sig_train[:, -1, j], 
                          c=data_train.index.hour[(settings['T']-1):])

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(15,15), constrained_layout=True)

day_number = 288
count = 4
for i in range(4):
    for j in range(4):
        axes[i,j].scatter(z_sig_train[count*day_number:(count+1)*day_number, -1, i], 
                          z_sig_train[count*day_number:(count+1)*day_number, -1, j], 
                          c=data_train.index.hour[(settings['T']-1)+count*day_number:(settings['T']-1)+(count+1)*day_number])

### $z$

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(15,15), constrained_layout=True)

for i in range(4):
    for j in range(4):
        axes[i,j].scatter(z_train[:, -1, i], 
                          z_train[:, -1, j], 
                          c=data_train.index.hour[(settings['T']-1):])

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(15,15), constrained_layout=True)

day_number = 288
count = 4
for i in range(4):
    for j in range(4):
        axes[i,j].scatter(z_train[count*day_number:(count+1)*day_number, -1, i], 
                          z_train[count*day_number:(count+1)*day_number, -1, j], 
                          c=data_train.index.hour[(settings['T']-1)+count*day_number:(settings['T']-1)+(count+1)*day_number])